In [41]:
from src.conv_onet.models.decoder_copy import NICE
import torch
import torch.nn as nn
import torch.nn.functional as F

In [42]:
grid_coarse = torch.zeros([1, 32, 3, 4, 6])
grid_middle = torch.zeros([1, 32, 10, 12, 20])
grid_fine = torch.zeros([1, 32, 21, 25, 41])
grid_color = torch.zeros([1, 32, 21, 25, 41])

In [43]:
bound = torch.Tensor([[0.0,6.5],[0.0,4.0],[0,3.5]])

In [5]:
coarse_len = 2
middle_len = 0.32
fine_len = 0.16

In [44]:
c_grid = dict({"grid_coarse": grid_coarse, "grid_middle": grid_middle, 
               "grid_fine": grid_fine, "grid_color": grid_color})

In [45]:
model = NICE(coarse = True)

In [108]:
x_range = torch.arange(bound[0][0], bound[0][1], middle_len)
y_range = torch.arange(bound[1][0], bound[1][1], middle_len)
z_range = torch.arange(bound[2][0], bound[2][1], middle_len)

In [110]:
p_list = []
for i in x_range:
    for j in y_range:
        for k in z_range:
            p_list.append([i, j, k])

p_tensor = torch.Tensor(p_list)
torch.sigmoid(10*model.forward(p_tensor, c_grid, 'middle')[:, -1])


tensor([1.1843e-03, 1.3579e-05, 5.0613e-03,  ..., 2.5837e-01, 8.5794e-01,
        1.8182e-07], grad_fn=<SigmoidBackward>)

In [8]:
p = torch.Tensor([[0,0,0], [0,0,0]])

In [59]:
feature_0 = torch.arange(0, 10, 0.1)
test_prob = []
for fea in feature_0:
    grid_middle = torch.zeros([1, 32, 10, 12, 20])
    grid_middle[0, 1, 0, 0, 0] = fea
    c_grid = dict({"grid_coarse": grid_coarse, "grid_middle": grid_middle, 
                "grid_fine": grid_fine, "grid_color": grid_color})
    prob = model.forward(p, c_grid, 'middle')[:, -1][0]
    if prob >-0.1 and prob <0.1:
        test_prob.append([grid_middle[0, 1, 0, 0, 0], prob])

In [20]:
test_prob

[]

In [36]:
import numpy as np

In [ ]:
#  torch.sigmoid(10*raw[..., -1])

In [106]:
from torch.autograd import Variable
p = torch.Tensor([[0,0,0], [0,1,0]])
middle_grid_para = Variable(torch.zeros(32), requires_grad = True)
optimizer = torch.optim.Adam([{'params': middle_grid_para, 'lr': 0.01}])
loss1 = torch.Tensor([3]).pow(2).sum()
for n in range(70):

    optimizer.zero_grad()
    grid_middle = torch.zeros([1, 32, 10, 12, 20])
    grid_middle[0, :, 0, 0, 0] = middle_grid_para
    c_grid = dict({"grid_coarse": grid_coarse, "grid_middle": grid_middle, 
               "grid_fine": grid_fine, "grid_color": grid_color})
    prob = torch.sigmoid(10*model.forward(p, c_grid, 'middle')[:, -1][0])
    loss = (prob).pow(2).sum()
    print(loss)
    if n>1 and loss.item()<loss1.item():
        print(1)
        final_middle = middle_grid_para
        loss1 = loss
    loss.backward()
    optimizer.step()


tensor(0.0694, grad_fn=<SumBackward0>)
tensor(0.0209, grad_fn=<SumBackward0>)
tensor(0.0065, grad_fn=<SumBackward0>)
1


AttributeError: 'Tensor' object has no attribute 'copy'

In [103]:

final_middle

tensor([-0.1057, -0.0879, -0.0830, -0.0849,  0.1114, -0.1169,  0.0843, -0.0727,
         0.0768, -0.0847,  0.0845, -0.0739,  0.0936,  0.0854, -0.0775, -0.0855,
        -0.0856, -0.0856, -0.0860, -0.0857,  0.0826,  0.0815, -0.0742,  0.0809,
        -0.0701,  0.0875, -0.0843, -0.0851, -0.0852,  0.0921,  0.0957, -0.0855],
       requires_grad=True)

In [107]:
grid_middle = torch.zeros([1, 32, 10, 12, 20])
grid_middle[0, :, 0, 0, 0] = final_middle
c_grid = dict({"grid_coarse": grid_coarse, "grid_middle": grid_middle, 
            "grid_fine": grid_fine, "grid_color": grid_color})
torch.sigmoid(10*model.forward(p, c_grid, 'middle')[:, -1])

tensor([1.1843e-03, 2.5747e-11], grad_fn=<SigmoidBackward>)

In [105]:
p

tensor([[0., 0., 0.],
        [0., 1., 0.]])